In [10]:
import json
import requests

In [2]:
search_template = """
{
   "query":{
      "function_score":{
         "functions":[
            {
               "exp":{
                  "issued.date":{
                     "offset":"30d",
                     "scale":"1800d"
                  }
               }
            }
         ],
         "score_mode":"sum",
         "query":{
            "bool":{
               "must":[
                  {
                     "multi_match":{
                        "query":"SEARCH_TERM",
                        "minimum_should_match":"75%",
                        "fields":[
                           "title^2",
                           "subject^1.5",
                           "description^1.5",
                           "all_fields_for_freetext"
                        ]
                     }
                  }
               ],
               "filter":[
                  {
                     "constant_score":{
                        "filter":{
                           "bool":{
                              "should":[
                                 {
                                    "bool":{
                                       "must_not":{
                                          "exists":{
                                             "field":"expires"
                                          }
                                       }
                                    }
                                 },
                                 {
                                    "range":{
                                       "expires":{
                                          "gte":"2022-03-16T15:59:50Z"
                                       }
                                    }
                                 }
                              ]
                           }
                        }
                     }
                  },
                  {
                     "bool":{
                        "should":[
                           {
                              "range":{
                                 "readingTime":{
                                    
                                 }
                              }
                           }
                        ],
                        "minimum_should_match":1
                     }
                  },
                  {
                     "bool":{
                        "should":[
                           {
                              "range":{
                                 "issued.date":{
                                    
                                 }
                              }
                           }
                        ],
                        "minimum_should_match":1
                     }
                  },
                  {
                     "bool":{
                        "should":[
                           {
                              "term":{
                                 "language":"en"
                              }
                           }
                        ],
                        "minimum_should_match":1
                     }
                  },
                  {
                     "term":{
                        "hasWorkflowState":"published"
                     }
                  },
                  {
                     "constant_score":{
                        "filter":{
                           "range":{
                              "issued.date":{
                                 "lte":"2022-03-16T15:59:52Z"
                              }
                           }
                        }
                     }
                  }
               ]
            }
         }
      }
   },
   "size":SIZE,
   "from":0,
   "track_total_hits":true,
   "explain":false
}
"""

In [3]:
_search_terms = """ 
Which car manufacturer has the lowest emissions?
Which transport mode has the lowest pollution?
How much have new cars co2 emissions decreased in 2020?
How many premature deaths are attributed to PM2.5?
How many people are affected by noise pollution?
How many protected sites do we have in Europe?
What is the cumulative surface area of the Natura 2000?
What percentage of europe population is connected to waste water treatement?
How is the status of fish species in Europe?
How is the qualtiy of bathing water in europe?
What is BISE?
What is PFAS?
How does PFAS get into human body?
How far does transport sector pollute in GHG total emissions?
How much does transport contributes to GHG emissions in Eu?
What transport mode is best for the environment?
What EU legislation safeguards our water?
What is land accounting?
What countries had the highest land take in the EEA-39 between 2000 and 2018?
Why is urban sprawl bad?
What are controlled substances?
What year did car manufacturers meet their binding emissions target?
What is the percentage of surface water bodies with less than good status?
What is the most common pollutant in water bodies in Europe?
Where can I access greenhouse gas data?
What is the trend on greenhouse gas emissions from transport?
What are ecosystem services?
What does urban environmental sustainability mean?
How to transition to urban environmental sustainability?
What are the urban environmental sustainability objectives?
What is the trend in waste generation?
How much clothing is wasted per person?
How much municipal waste is recycled?
How much hazardous waste is exported?
How many more trees should be planted in the future?
How many people are exposed to air pollution?
How much of europe's population is exposed to fine particulate matter?
How much of land and marine areas are protected?
How to transition to urban environmental sustainability?
What are the urban environmental sustainability objectives?
What is the trend of waste generation in Europe?
What is the trend in waste generation?
How much clothing is wasted per person?
How much municipal waste is recycled?
How much hazardous waste is exported?
How many more trees should be planted in the future?
How many people are exposed to air pollution?
How much of Europe's population is exposed to fine particulate matter?
How much of land and marine areas are protected?
What are the effects of particulate matter on our health?
Which type of fuel is mostly sold?
Which countries use most renewable energy?
"""
search_terms = list(filter(lambda l: l.strip(), _search_terms.split("\n")))
search_terms

['Which car manufacturer has the lowest emissions?',
 'Which transport mode has the lowest pollution?',
 'How much have new cars co2 emissions decreased in 2020?',
 'How many premature deaths are attributed to PM2.5?',
 'How many people are affected by noise pollution?',
 'How many protected sites do we have in Europe?',
 'What is the cumulative surface area of the Natura 2000?',
 'What percentage of europe population is connected to waste water treatement?',
 'How is the status of fish species in Europe?',
 'How is the qualtiy of bathing water in europe?',
 'What is BISE?',
 'What is PFAS?',
 'How does PFAS get into human body?',
 'How far does transport sector pollute in GHG total emissions?',
 'How much does transport contributes to GHG emissions in Eu?',
 'What transport mode is best for the environment?',
 'What EU legislation safeguards our water?',
 'What is land accounting?',
 'What countries had the highest land take in the EEA-39 between 2000 and 2018?',
 'Why is urban sprawl

In [16]:
host = "http://10.120.10.131:57664/data_searchui/_search"

def get_text(search_term, size=20):
    text = search_template.replace("SEARCH_TERM", search_term).replace("SIZE", str(size))
    resp = requests.post(host, json=json.loads(text))
    hits = resp.json()['hits']['hits']
    return [h['_source']['fulltext'] for h in hits]

In [18]:
for i, s in enumerate(search_terms):
    texts = get_text(s)
    fname = f"data/{i}.json"
    d = dict(query=s, texts=texts)
    with open(fname, 'w') as f:
        json.dump(d, f)